In [6]:
import tensorflow as tf
from tensorflow import keras

In [7]:
tf.__version__

'2.10.0'

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/training_set",
    image_size=(64,64),
    batch_size=32,
    label_mode="binary",
    validation_split=0.2,
    subset="training",
    seed=42
)
#imagedatagenerator old api using image_dataset_from_directory beter and faster need to add argumentation seperately
val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/training_set",
    image_size=(64,64),
    batch_size=32,
    label_mode="binary",
    validation_split=0.2,
    subset="validation",
    seed=42
)
# ⚡️ Normalization (equivalent to rescale=1./255)
normalization_layer = keras.layers.Rescaling(1./255)

# ⚡️ Augmentation layers (equivalent to shear, zoom, flip)
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomZoom(0.2),
    keras.layers.RandomRotation(0.2),  # kinda like shear
])
class_names = train_ds.class_names #to get class name before tf starts and get indices from subfolders

print("Class names:", class_names)
# Apply normalization + augmentation
train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x), training=True), y))
val_ds   = val_ds.map(lambda x, y: (normalization_layer(x), y))

train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds   = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)


Found 8000 files belonging to 2 classes.
Using 6400 files for training.
Found 8000 files belonging to 2 classes.
Using 1600 files for validation.
Class names: ['cats', 'dogs']


In [9]:
cnn = tf.keras.models.Sequential()

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [11]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [12]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [13]:
cnn.add(tf.keras.layers.Flatten())

In [14]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [15]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [16]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [17]:
cnn.fit(x = train_ds, validation_data = val_ds, epochs = 25)

Epoch 1/25
200/200 [==============================] - 14s 40ms/step - loss: 0.6906 - accuracy: 0.5431 - val_loss: 0.6556 - val_accuracy: 0.6394
Epoch 2/25
200/200 [==============================] - 9s 44ms/step - loss: 0.6623 - accuracy: 0.6131 - val_loss: 0.6321 - val_accuracy: 0.6538
Epoch 3/25
200/200 [==============================] - 9s 44ms/step - loss: 0.6340 - accuracy: 0.6459 - val_loss: 0.6383 - val_accuracy: 0.6469
Epoch 4/25
200/200 [==============================] - 9s 45ms/step - loss: 0.6037 - accuracy: 0.6697 - val_loss: 0.5536 - val_accuracy: 0.7169
Epoch 5/25
200/200 [==============================] - 10s 49ms/step - loss: 0.5992 - accuracy: 0.6705 - val_loss: 0.5452 - val_accuracy: 0.7262
Epoch 6/25
200/200 [==============================] - 9s 42ms/step - loss: 0.5753 - accuracy: 0.6944 - val_loss: 0.5898 - val_accuracy: 0.6988
Epoch 7/25
200/200 [==============================] - 8s 39ms/step - loss: 0.5687 - accuracy: 0.7014 - val_loss: 0.6590 - val_accuracy: 0.65

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
# Binary output → sigmoid activation, so we check probability > 0.5
if result[0][0] > 0.5:
    prediction = class_names[1]  # 'dogs'
else:
    prediction = class_names[0]  # 'cats'

print("Prediction:", prediction)


1/1 [==============================] - 0s 28ms/step
Prediction: dogs


In [ ]:
print(prediction)

dogs
